# AER - Access Entitlement Review Reporter

從 SharePoint 讀取審核人員的 Excel 回應，產生缺漏報告與合併報告。

## Cell 1: 設定與認證

In [34]:
# === 設定與認證 ===
import os
import logging
from datetime import datetime
from dotenv import load_dotenv
from msal import ConfidentialClientApplication
import requests

load_dotenv()

# === Logging 設定 ===
os.makedirs("output", exist_ok=True)
log_file = f"output/aer_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

logger = logging.getLogger("aer")
logger.handlers.clear()  # 避免重複執行時累積 handler
logger.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')

ch = logging.StreamHandler()
ch.setFormatter(formatter)
logger.addHandler(ch)

fh = logging.FileHandler(log_file)
fh.setFormatter(formatter)
logger.addHandler(fh)

# === Azure AD 設定 ===
TENANT_ID = os.getenv("AZURE_TENANT_ID")
CLIENT_ID = os.getenv("AZURE_CLIENT_ID")
CLIENT_SECRET = os.getenv("AZURE_CLIENT_SECRET")

# === SharePoint 設定 ===
SHAREPOINT_HOST = os.getenv("SHAREPOINT_HOST", "davidshih.sharepoint.com")
SITE_NAME = os.getenv("SITE_NAME", "aer")
APP_NAME = "2025 Entitlement Review/Q4/AVIDXCHANGE-TEST"
BASE_PATH = APP_NAME

# === Email 設定 ===
SENDER_EMAIL = os.getenv("SENDER_EMAIL")  # e.g., "noreply@company.com"

# === 取得 Access Token ===
app = ConfidentialClientApplication(
    CLIENT_ID,
    authority=f"https://login.microsoftonline.com/{TENANT_ID}",
    client_credential=CLIENT_SECRET
)
token_result = app.acquire_token_for_client(scopes=["https://graph.microsoft.com/.default"])

if "access_token" not in token_result:
    logger.error(f"認證失敗: {token_result.get('error_description', 'Unknown error')}")
    raise Exception("Azure AD 認證失敗")

headers = {"Authorization": f"Bearer {token_result['access_token']}"}
logger.info(f"Azure AD 認證成功，處理 app: {APP_NAME}")
logger.info(f"Log 檔案: {log_file}")

2025-12-29 13:28:38,423 | INFO | Azure AD 認證成功，處理 app: 2025 Entitlement Review/Q4/AVIDXCHANGE-TEST
2025-12-29 13:28:38,424 | INFO | Log 檔案: output/aer_20251229_132838.log


## Cell 2: SharePoint 函數

In [35]:
# === Cell 2: SharePoint 函數 (Level 2 升級版) ===

def get_site_id(site_name: str) -> str:
    """取得 SharePoint site ID"""
    url = f"https://graph.microsoft.com/v1.0/sites/{SHAREPOINT_HOST}:/sites/{site_name}"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()["id"]


def list_folders(site_id: str, path: str) -> list[dict]:
    """列出資料夾下的所有子資料夾（含 webUrl）"""
    # 注意：path 已經是相對路徑了
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root:/{path}:/children"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return [
        {"name": item["name"], "webUrl": item.get("webUrl", "")}
        for item in resp.json().get("value", [])
        if item.get("folder")
    ]


def list_excel_files(site_id: str, folder_path: str) -> list[dict]:
    """列出資料夾下的 Excel 檔案（回傳 id, name, lastModified）"""
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root:/{folder_path}:/children"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    
    files = []
    for item in resp.json().get("value", []):
        if item["name"].endswith(".xlsx"):
            files.append({
                "id": item["id"],  # <--- 關鍵：抓 ID
                "name": item["name"],
                "lastModifiedDateTime": item.get("lastModifiedDateTime"),
                "webUrl": item.get("webUrl")
            })
            
    # 按最後修改時間排序（最新的在前）
    return sorted(files, key=lambda f: f.get("lastModifiedDateTime", ""), reverse=True)


def download_file(site_id: str, file_path: str) -> bytes:
    """下載檔案內容"""
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root:/{file_path}:/content"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.content


def get_file_audit_log(site_id: str, file_id: str) -> str:
    """
    [新增] 取得檔案版本歷史 (Audit Log)
    回傳格式: "時間 - 人員 (動作)\n時間 - 人員 (動作)..."
    """
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/items/{file_id}/versions"
    resp = requests.get(url, headers=headers)
    
    if resp.status_code != 200:
        return "無法取得版本紀錄"

    versions = resp.json().get("value", [])
    logs = []
    
    for v in versions:
        # 時間格式化 (e.g., 2025-12-29T10:00:00Z -> 2025-12-29 10:00:00)
        mod_time = v.get("lastModifiedDateTime", "")[:19].replace("T", " ")
        
        # 抓修改人 (優先抓 User DisplayName，抓不到就抓 Application Name)
        user_info = v.get("lastModifiedBy", {}).get("user", {})
        app_info = v.get("lastModifiedBy", {}).get("application", {})
        
        actor = user_info.get("displayName") or app_info.get("displayName") or "Unknown System"
        
        # 判斷是否為最新版本 (Graph API 的版本通常是 1.0, 2.0...)
        ver_num = v.get("id", "")
        
        logs.append(f"{mod_time} - {actor} (v{ver_num})")
        
    return "\n".join(logs) # 用換行符號串接，方便在 Excel 同一格顯示


# 測試連線
try:
    site_id = get_site_id(SITE_NAME)
    logger.info(f"SharePoint 連線成功，Site ID: {site_id[:20]}...")
except Exception as e:
    logger.error(f"SharePoint 連線失敗: {e}")
    raise

2025-12-29 13:28:43,422 | INFO | SharePoint 連線成功，Site ID: davidshih.sharepoint...


## Cell 3: Excel 讀取

In [36]:
# === Excel 讀取（處理 AutoFilter visible rows）===
from openpyxl import load_workbook
from io import BytesIO

# 欄位名稱設定
COL_REVIEWER = "Reviewer"
COL_RESPONSE = "Reviewer's Response"
COL_DETAILS = "Details of Access change"


def read_visible_rows(excel_bytes: bytes, reviewer_name: str, file_name: str, folder_url: str) -> list[dict]:
    """
    讀取 Excel 的 visible rows（考慮 AutoFilter）
    
    Args:
        excel_bytes: Excel 檔案的 bytes
        reviewer_name: 審核人名稱（用於 fallback 過濾）
        file_name: 檔案名稱（追蹤用）
        folder_url: SharePoint 資料夾 URL（可點擊）
    
    Returns:
        list of ReviewerResponse dict
    """
    wb = load_workbook(BytesIO(excel_bytes))
    ws = wb.active
    
    # 找出欄位索引（假設第一行是標題）
    header_row = [cell.value for cell in ws[1]]
    col_map = {str(name).strip(): idx for idx, name in enumerate(header_row) if name}
    
    # 檢查必要欄位
    reviewer_col = col_map.get(COL_REVIEWER)
    response_col = col_map.get(COL_RESPONSE)
    details_col = col_map.get(COL_DETAILS)
    
    if reviewer_col is None or response_col is None:
        raise ValueError(f"找不到必要欄位。現有欄位: {list(col_map.keys())}")
    
    # 檢查是否有 hidden rows（表示有 AutoFilter）
    has_filter = any(
        ws.row_dimensions[i].hidden 
        for i in range(2, ws.max_row + 1) 
        if i in ws.row_dimensions
    )
    
    results = []
    for row_idx in range(2, ws.max_row + 1):
        # 如果有 filter，跳過被隱藏的行
        if has_filter and ws.row_dimensions.get(row_idx) and ws.row_dimensions[row_idx].hidden:
            continue
        
        row = [cell.value for cell in ws[row_idx]]
        
        # 取得欄位值
        reviewer = row[reviewer_col] if reviewer_col < len(row) else None
        response = row[response_col] if response_col < len(row) else None
        details = row[details_col] if details_col is not None and details_col < len(row) else None
        
        # 如果沒有 filter，用欄位值過濾
        if not has_filter:
            if reviewer is None or str(reviewer).strip().lower() != reviewer_name.lower():
                continue
        
        # 判斷是否缺漏
        is_missing = response is None or str(response).strip() == ""
        
        results.append({
            "reviewer": reviewer_name,
            "response": response,
            "details": details,
            "is_missing": is_missing,
            "row_number": row_idx,
            "file_name": file_name,
            "folder_url": folder_url
        })
    
    wb.close()
    return results


logger.info("Excel 讀取函數已載入")

2025-12-29 13:28:46,895 | INFO | Excel 讀取函數已載入


## Cell 4: 資料收集

In [37]:
# === Cell 4: 資料收集 (含 Audit Log) ===
import pandas as pd

# 確保 BASE_PATH 是正確的相對路徑 (不含 Shared Documents)
# 如果您在 Cell 1 已經修好了，這裡直接用即可
folders = list_folders(site_id, BASE_PATH)
logger.info(f"找到 {len(folders)} 個審核人資料夾")

all_responses = []
errors = []

for folder in folders:
    reviewer_name = folder["name"]
    folder_url = folder["webUrl"]
    # 組合路徑: Base path + Reviewer Folder
    folder_path = f"{BASE_PATH}/{reviewer_name}"
    
    try:
        logger.info(f"處理審核人: {reviewer_name}")
        
        # 列出 Excel 檔案
        excel_files = list_excel_files(site_id, folder_path)
        
        # 找到包含審核人名字的 Excel 檔案
        target_files = [f for f in excel_files if reviewer_name.lower() in f["name"].lower()]
        
        if not target_files:
            logger.warning(f"  找不到包含 '{reviewer_name}' 的 Excel 檔案")
            errors.append({"reviewer": reviewer_name, "error": "找不到審核檔案", "folder_url": folder_url})
            continue
        
        # 讀取最新的檔案
        target_file = target_files[0]
        file_name = target_file["name"]
        file_id = target_file["id"]
        # 注意：路徑要 encode 處理空格，但 requests 會自動幫忙，這裡直接串字串
        file_download_path = f"{folder_path}/{file_name}"
        
        logger.info(f"  讀取檔案: {file_name}")
        
        # 🔥 [關鍵動作] 抓取 Audit Log (版本歷史)
        audit_log_str = get_file_audit_log(site_id, file_id)
        
        # 下載內容
        excel_bytes = download_file(site_id, file_download_path)
        
        # 解析 Excel
        rows = read_visible_rows(excel_bytes, reviewer_name, file_name, folder_url)
        
        # 把 Audit Log 塞給每一行 (讓每一筆資料都帶有這個檔案的修改歷史)
        for row in rows:
            row["Audit_History"] = audit_log_str
            row["Last_Modified"] = target_file["lastModifiedDateTime"]
        
        all_responses.extend(rows)
        
        missing_count = sum(1 for r in rows if r["is_missing"])
        logger.info(f"  完成: {len(rows)} 筆記錄，{missing_count} 筆缺漏")
        
    except Exception as e:
        logger.error(f"  處理失敗: {e}")
        # 印出 traceback 方便除錯
        import traceback
        traceback.print_exc() 
        errors.append({"reviewer": reviewer_name, "error": str(e), "folder_url": folder_url})

# 轉成 DataFrame
df = pd.DataFrame(all_responses)
logger.info(f"\n=== 收集完成 ===")
logger.info(f"總計: {len(all_responses)} 筆記錄")
logger.info(f"缺漏: {df['is_missing'].sum() if len(df) > 0 else 0} 筆")
logger.info(f"錯誤: {len(errors)} 個審核人")

2025-12-29 13:28:50,271 | INFO | 找到 3 個審核人資料夾
2025-12-29 13:28:50,272 | INFO | 處理審核人: Bob O'Brien
2025-12-29 13:28:50,604 | INFO |   讀取檔案: Bob O'Brien_Access_Review_2025Q4.xlsx
2025-12-29 13:28:51,493 | INFO |   完成: 3 筆記錄，0 筆缺漏
2025-12-29 13:28:51,493 | INFO | 處理審核人: Jane Smith
2025-12-29 13:28:51,964 | INFO |   讀取檔案: Jane Smith_Access_Review_2025Q4.xlsx
2025-12-29 13:28:52,935 | INFO |   完成: 5 筆記錄，1 筆缺漏
2025-12-29 13:28:52,936 | INFO | 處理審核人: Joe Petti
2025-12-29 13:28:54,222 | INFO |   讀取檔案: Joe Petti_Access_Review_2025Q4.xlsx
2025-12-29 13:28:55,440 | INFO |   完成: 4 筆記錄，2 筆缺漏
2025-12-29 13:28:55,442 | INFO | 
=== 收集完成 ===
2025-12-29 13:28:55,442 | INFO | 總計: 12 筆記錄
2025-12-29 13:28:55,443 | INFO | 缺漏: 3 筆
2025-12-29 13:28:55,443 | INFO | 錯誤: 0 個審核人


## Cell 5: 互動式狀態表格

In [39]:
# === 互動式狀態表格 ===
from IPython.display import display, HTML

def create_status_table(df: pd.DataFrame) -> pd.DataFrame:
    """建立互動式狀態表格，返回 summary DataFrame"""
    
    if len(df) == 0:
        display(HTML("<p style='color: red;'>沒有資料可顯示</p>"))
        return pd.DataFrame()
    
    # 計算每個審核人的狀態
    summary = df.groupby(["reviewer", "folder_url"]).agg(
        total=("reviewer", "count"),
        missing=("is_missing", "sum")
    ).reset_index()
    
    summary["status"] = summary.apply(
        lambda r: "🔴 未完成" if r["missing"] > 0 else "🟢 完成", axis=1
    )
    summary["completion"] = ((summary["total"] - summary["missing"]) / summary["total"] * 100).round(1)
    
    # 建立 HTML 表格（可點擊連結到 SharePoint）
    def make_row(r):
        bg_color = "#ffcccc" if r["missing"] > 0 else "#ccffcc"
        return f"""
        <tr style="background-color: {bg_color};">
            <td><a href="{r['folder_url']}" target="_blank">{r['reviewer']}</a></td>
            <td>{r['status']}</td>
            <td>{r['completion']}%</td>
            <td>{int(r['missing'])} / {int(r['total'])}</td>
        </tr>
        """
    
    # 排序：未完成的在前
    summary_sorted = summary.sort_values(["missing", "reviewer"], ascending=[False, True])
    
    html = f"""
    <style>
        .status-table {{ border-collapse: collapse; width: 100%; }}
        .status-table th, .status-table td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
        .status-table th {{ background-color: #4CAF50; color: white; }}
        .status-table a {{ color: #0066cc; text-decoration: none; }}
        .status-table a:hover {{ text-decoration: underline; }}
    </style>
    <h3>審核狀態總覽 - {APP_NAME}</h3>
    <table class="status-table">
        <tr>
            <th>審核人</th>
            <th>狀態</th>
            <th>完成率</th>
            <th>缺漏/總數</th>
        </tr>
        {''.join(summary_sorted.apply(make_row, axis=1))}
    </table>
    <p><b>總計:</b> {len(summary)} 位審核人, {int(summary['missing'].sum())} 筆缺漏</p>
    """
    
    display(HTML(html))
    return summary


# 顯示狀態表格
summary_df = create_status_table(df)

審核人,狀態,完成率,缺漏/總數
Joe Petti,🔴 未完成,50.0%,2 / 4
Jane Smith,🔴 未完成,80.0%,1 / 5
Bob O'Brien,🟢 完成,100.0%,0 / 3


## Cell 6: 發送提醒郵件

In [ ]:
# === 發送提醒郵件 ===
import ipywidgets as widgets
from urllib.parse import quote

def get_user_email(display_name: str) -> str | None:
    """從 Azure AD 查詢使用者 email"""
    # URL encode 處理特殊字元
    encoded_name = quote(display_name)
    user_url = f"https://graph.microsoft.com/v1.0/users?$filter=displayName eq '{encoded_name}'"
    
    resp = requests.get(user_url, headers=headers)
    if resp.status_code != 200:
        logger.warning(f"查詢使用者失敗: {resp.text}")
        return None
    
    users = resp.json().get("value", [])
    if not users:
        logger.warning(f"找不到 {display_name} 的 email")
        return None
    
    return users[0].get("mail")


def send_reminder_email(reviewer_name: str, missing_count: int, folder_url: str, deadline: str = None) -> bool:
    """發送提醒郵件給審核人"""
    
    if not SENDER_EMAIL:
        logger.error("未設定 SENDER_EMAIL")
        return False
    
    recipient_email = get_user_email(reviewer_name)
    if not recipient_email:
        return False
    
    # 用指定發送者（Application 認證不能用 /me）
    mail_url = f"https://graph.microsoft.com/v1.0/users/{SENDER_EMAIL}/sendMail"
    
    deadline_text = f"<p><b>截止日期:</b> {deadline}</p>" if deadline else ""
    
    mail_body = {
        "message": {
            "subject": f"[提醒] Access Review 待審核 - {missing_count} 筆待處理",
            "body": {
                "contentType": "HTML",
                "content": f"""
                <p>Hi {reviewer_name},</p>
                <p>您有 <b>{missing_count}</b> 筆 Access Review 記錄待審核。</p>
                {deadline_text}
                <p>請點擊以下連結完成審核：</p>
                <p><a href="{folder_url}">{folder_url}</a></p>
                <br>
                <p>謝謝！</p>
                """
            },
            "toRecipients": [{"emailAddress": {"address": recipient_email}}]
        }
    }
    
    resp = requests.post(mail_url, headers={**headers, "Content-Type": "application/json"}, json=mail_body)
    
    if resp.status_code == 202:
        logger.info(f"已發送提醒郵件給 {reviewer_name} ({recipient_email})")
        return True
    else:
        logger.error(f"郵件發送失敗: {resp.status_code} - {resp.text}")
        return False


def create_reminder_ui(summary_df: pd.DataFrame):
    """建立發送提醒的互動式 UI"""
    
    if len(summary_df) == 0:
        display(HTML("<p>沒有資料</p>"))
        return
    
    # 只顯示有缺漏的審核人
    missing_df = summary_df[summary_df["missing"] > 0]
    
    if len(missing_df) == 0:
        display(HTML("<p style='color: green;'>✅ 所有審核人都已完成！</p>"))
        return
    
    # 建立 checkbox
    checkboxes = {}
    for _, row in missing_df.iterrows():
        name = row["reviewer"]
        label = f"{name} ({int(row['missing'])} 筆缺漏)"
        checkboxes[name] = widgets.Checkbox(value=False, description=label, style={'description_width': 'initial'})
    
    # 全選按鈕
    select_all = widgets.Checkbox(value=False, description="全選", style={'description_width': 'initial'})
    
    def on_select_all_change(change):
        for cb in checkboxes.values():
            cb.value = change['new']
    
    select_all.observe(on_select_all_change, names='value')
    
    # 截止日期
    deadline_input = widgets.Text(value="", placeholder="例如: 2024-01-15", description="截止日期:")
    
    # 發送按鈕
    send_btn = widgets.Button(description="發送提醒郵件", button_style="warning")
    output = widgets.Output()
    
    def on_send(b):
        with output:
            output.clear_output()
            deadline = deadline_input.value.strip() or None
            sent_count = 0
            for name, cb in checkboxes.items():
                if cb.value:
                    row = missing_df[missing_df["reviewer"] == name].iloc[0]
                    if send_reminder_email(name, int(row["missing"]), row["folder_url"], deadline):
                        sent_count += 1
            print(f"\n已發送 {sent_count} 封提醒郵件")
    
    send_btn.on_click(on_send)
    
    # 顯示 UI
    display(HTML("<h3>發送提醒郵件</h3>"))
    display(select_all)
    display(widgets.VBox(list(checkboxes.values())))
    display(deadline_input)
    display(send_btn)
    display(output)


# 建立提醒 UI
create_reminder_ui(summary_df)

## Cell 7: 產生報告

In [41]:
# === Cell 7: 產生報告 (含格式美化) ===
from openpyxl import load_workbook
from openpyxl.styles import Alignment, PatternFill

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
safe_app_name = APP_NAME.replace("/", "_").replace("\\", "_")

def format_excel_layout(file_path: str):
    """
    [美化函數] 自動調整 Excel 格式：
    1. 開啟 'Audit_History' 和 'details' 的自動換行 (Wrap Text)
    2. 調整欄寬，避免內容擠在一起
    3. 將內容設為靠上對齊 (Vertical Top)
    """
    try:
        wb = load_workbook(file_path)
        ws = wb.active
        
        # 定義要處理的欄位名稱
        wrap_columns = ["Audit_History", "Details of Access change", "details"]
        
        # 找出這些欄位在哪一列 (Column Index)
        col_indices = {}
        header_row = next(ws.iter_rows(min_row=1, max_row=1, values_only=True))
        for idx, col_name in enumerate(header_row):
            if col_name in wrap_columns:
                col_indices[idx + 1] = col_name # Excel index starts from 1
        
        # 設定樣式
        align_style = Alignment(wrap_text=True, vertical='top')
        
        # 調整欄寬 (稍微拉寬一點給 Audit Log 用)
        for col_idx in col_indices:
            col_letter = ws.cell(row=1, column=col_idx).column_letter
            ws.column_dimensions[col_letter].width = 50 
        
        # 逐行套用格式
        for row in ws.iter_rows(min_row=2): # 跳過標題列
            for col_idx in col_indices:
                cell = row[col_idx - 1] # list index starts from 0
                cell.alignment = align_style
                
        wb.save(file_path)
        logger.info(f"✨ 已美化 Excel 格式 (自動換行): {file_path}")
        
    except Exception as e:
        logger.warning(f"Excel 格式美化失敗 (但不影響檔案內容): {e}")


# === 主要邏輯 ===
if len(df) > 0:
    # 定義輸出的欄位順序
    export_cols = [
        "reviewer", "response", "details", "is_missing", 
        "Audit_History", "Last_Modified", 
        "row_number", "file_name", "folder_url"
    ]
    
    # 1. 缺漏報告
    missing_df = df[df["is_missing"]].copy()
    missing_file_xlsx = f"output/missing_responses_{safe_app_name}_{timestamp}.xlsx"
    
    available_cols = [c for c in export_cols if c in missing_df.columns]
    missing_df[available_cols].to_excel(missing_file_xlsx, index=False)
    format_excel_layout(missing_file_xlsx) # <--- 呼叫美化函數
    
    # 2. 合併報告
    consolidated_file_xlsx = f"output/consolidated_report_{safe_app_name}_{timestamp}.xlsx"
    
    available_cols = [c for c in export_cols if c in df.columns]
    df[available_cols].to_excel(consolidated_file_xlsx, index=False)
    format_excel_layout(consolidated_file_xlsx) # <--- 呼叫美化函數
    
    display(HTML(f"""
    <h3>報告已產生 (含 Audit 格式美化)</h3>
    <ul>
        <li>缺漏報告: <code>{missing_file_xlsx}</code></li>
        <li>合併報告: <code>{consolidated_file_xlsx}</code></li>
    </ul>
    """))

else:
    logger.warning("沒有資料可產生報告")
    display(HTML("<p style='color: orange;'>沒有資料可產生報告</p>"))

2025-12-29 13:33:14,010 | INFO | ✨ 已美化 Excel 格式 (自動換行): output/missing_responses_2025 Entitlement Review_Q4_AVIDXCHANGE-TEST_20251229_133313.xlsx


2025-12-29 13:33:14,024 | INFO | ✨ 已美化 Excel 格式 (自動換行): output/consolidated_report_2025 Entitlement Review_Q4_AVIDXCHANGE-TEST_20251229_133313.xlsx


## Cell 8: 錯誤報告

In [ ]:
# === 錯誤報告 ===

if errors:
    error_df = pd.DataFrame(errors)
    error_file = f"output/errors_{APP_NAME}_{timestamp}.csv"
    error_df.to_csv(error_file, index=False)
    
    logger.warning(f"有 {len(errors)} 個審核人處理失敗")
    
    # 顯示錯誤表格
    def make_error_row(r):
        return f"""
        <tr style="background-color: #fff3cd;">
            <td><a href="{r['folder_url']}" target="_blank">{r['reviewer']}</a></td>
            <td>{r['error']}</td>
        </tr>
        """
    
    html = f"""
    <h3>⚠️ 處理錯誤</h3>
    <table class="status-table">
        <tr>
            <th>審核人</th>
            <th>錯誤訊息</th>
        </tr>
        {''.join(error_df.apply(make_error_row, axis=1))}
    </table>
    <p>錯誤報告已儲存: <code>{error_file}</code></p>
    """
    display(HTML(html))
else:
    display(HTML("<p style='color: green;'>✅ 所有審核人都處理成功！</p>"))